## Домашнее задание по теме урока: Логистическая регрессия. Log Loss.
**Постараюсь реализовать многоклассовую логрегрессию (частный случай которой - 2 класса**)

***Соответствующими комметариями буду указывать на реализации кода, которые соответствуют номерам ДЗ***

In [61]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [9]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2,  1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

In [10]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [11]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

In [12]:
X_st

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  2.        ,  5.        ,  3.        ,
         0.        ,  5.        , 10.        ,  1.        ,  2.        ],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  2.        ,
         1.        ,  3.        ,  3.        ,  1.        ,  2.        ]])

In [13]:
'''функция для преобразования вектора индексов классов в матрицу, содержащий
one-hot веткор для каждого экземпляра. В нашем случае это два экземпляра,
но данная реализация может быть использованап по множество экземпляров'''
def to_one_hot(y):
    n_classes = y.max() + 1
    m = len(y)
    Y_one_hot = np.zeros((m, n_classes))
    Y_one_hot[np.arange(m), y] = 1
    return Y_one_hot

In [14]:
y

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

In [15]:
Y_train_one_hot = to_one_hot(y)
Y_train_one_hot

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.]])

In [53]:
#реализация функции softmax (как в scipy.special) (оно же, по сути, №3 домашки)
def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps, axis=1, keepdims=True)
    return exps / exp_sums

In [46]:
# определим количество входов и выходов для обучения
n_inputs = X_st.T.shape[1] # == 4 (3 признака плюс смещение)
n_outputs = len(np.unique(y))   # == 2 класса
n_inputs, n_outputs

(4, 2)

#### 1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

*при реализации просто добавлю epsilon = 1e-7, это избавит как от нулей, так и от nan*

In [132]:
eta = 0.01
n_iterations = 10001
m = len(X_st)
epsilon = 1e-7

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_st.T.dot(Theta)#транспонирую X_st (здесь везде далее), чтобы соответствовать стандартным формулам
    Y_proba = softmax(logits)
    loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
    error = Y_proba - Y_train_one_hot
    if iteration % 500 == 0:
        print(iteration, loss)
    gradients = 1/m * X_st.dot(error)
    Theta = Theta - eta * gradients

0 1.8849594396399738
500 0.5206043754780034
1000 0.4673373707015725
1500 0.4330359244906285
2000 0.40640762491548726
2500 0.3850671400483333
3000 0.36772381015948136
3500 0.3534427427482506
4000 0.34152086495490275
4500 0.3314307438324783
5000 0.3227771741632863
5500 0.3152628888808072
6000 0.3086626429852748
6500 0.3028041731654258
7000 0.2975543842871299
7500 0.2928093571827214
8000 0.28848710125660815
8500 0.28452226685361814
9000 0.2808622575899294
9500 0.27746434703806017
10000 0.27429352062892814


In [109]:
Theta

array([[ 3.89369171, -3.65092657],
       [ 1.60299608,  0.45776763],
       [ 0.79637459, -0.61327012],
       [-4.28314345,  2.33785836]])

In [111]:
#ДЗ 3 и 4
logits = X_st.T.dot(Theta)
Y_proba = softmax(logits) # это есть номер 3 домашки
y_predict = np.argmax(Y_proba, axis=1) # это есть номер 4 домашки
Y_proba,y_predict

(array([[0.66551552, 0.33448448],
        [0.78063863, 0.21936137],
        [0.01693944, 0.98306056],
        [0.99616443, 0.00383557],
        [0.29300652, 0.70699348],
        [0.6022559 , 0.3977441 ],
        [0.00626796, 0.99373204],
        [0.89222541, 0.10777459],
        [0.63242085, 0.36757915],
        [0.03441469, 0.96558531]]),
 array([0, 0, 1, 0, 1, 0, 1, 0, 0, 1], dtype=int64))

In [112]:
# ДЗ 5
accuracy_score = np.mean(y_predict == y)
accuracy_score

0.9

In [119]:
# ДЗ 5
table = np.array([sum(y_predict), len(y)-sum(y_predict), sum(y), len(y)-sum(y)]).reshape(2, 2)
confusion_df = pd.DataFrame(table, ['𝑎(𝑥)=+1', '𝑎(𝑥)=−1'], ['𝑦=+1', '𝑦=−1'])
confusion_df

,𝑦=+1,𝑦=−1
𝑎(𝑥)=+1,4,6
𝑎(𝑥)=−1,5,5


In [131]:
# ДЗ 5
Precission = confusion_df.iloc[0, 0]/(confusion_df.iloc[0, 0] + confusion_df.iloc[0, 1])
Recall = confusion_df.iloc[0, 0]/(confusion_df.iloc[0, 0] + confusion_df.iloc[1, 0])
F = (2*Precission*Recall)/(Precission+Recall)
Precission, Recall, F

(0.4, 0.4444444444444444, 0.4210526315789474)

## ДЗ 2 и 6

**Да модель могла переобучиться, потому что мы никак не ограничивали наши веса. Ниже я делаю реализацию регуляризации, а также прекращения градиентного спуска во избежание лишних иттераций (что соответствкет ДЗ 2 по сути)**

In [138]:
eta = 0.1 
n_iterations = 10001
m = len(X_st)
epsilon = 1e-7
alpha = 0.1  # гиперпараметр регуляризации
best_loss = np.infty

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_st.T.dot(Theta)
    Y_proba = softmax(logits)
    xentropy_loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
    l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
    loss = xentropy_loss + alpha * l2_loss
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_st.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]
    Theta = Theta - eta * gradients

    logits = X_st.T.dot(Theta)
    Y_proba = softmax(logits)
    xentropy_loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
    l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
    loss = xentropy_loss + alpha * l2_loss
    if iteration % 10 == 0:
        print(iteration, loss)
    if loss < best_loss:
        best_loss = loss
    else:
        print(iteration - 1, best_loss)
        print(iteration, loss, "early stopping!")
        break

0 1.4808548146474751
10 0.8368955129234252
20 0.7367632859953441
30 0.6994523807427235
40 0.6707961729201005
50 0.6471381032008142
60 0.62753995374513
70 0.6113386484563603
80 0.5979939325731315
90 0.587052865631403
100 0.5781321980737022
110 0.570906977541903
120 0.5651018741444419
130 0.5604839792822818
140 0.5568566176827491
150 0.5540539808210774
160 0.55193647780917
170 0.5503867264675005
180 0.549306114896192
190 0.5486118670116448
200 0.5482345487763649
210 0.5481159562038957
210 0.5481159562038957
211 0.5481163763633492 early stopping!


In [139]:
Theta

array([[ 0.69946563, -0.8870627 ],
       [ 0.34867677, -0.30854701],
       [-0.08761391,  0.25104296],
       [-0.87694978,  1.19983414]])

In [140]:
logits = X_st.T.dot(Theta)
Y_proba = softmax(logits) # это есть номер 3 домашки
y_predict = np.argmax(Y_proba, axis=1) # это есть номер 4 домашки
Y_proba,y_predict

(array([[0.62215683, 0.37784317],
        [0.58880584, 0.41119416],
        [0.25059346, 0.74940654],
        [0.95511667, 0.04488333],
        [0.2835108 , 0.7164892 ],
        [0.4090162 , 0.5909838 ],
        [0.15130916, 0.84869084],
        [0.77074878, 0.22925122],
        [0.63033017, 0.36966983],
        [0.21924832, 0.78075168]]),
 array([0, 0, 1, 0, 1, 1, 1, 0, 0, 1], dtype=int64))

In [141]:
accuracy_score = np.mean(y_predict == y)
accuracy_score

0.8

In [142]:
table = np.array([sum(y_predict), len(y)-sum(y_predict), sum(y), len(y)-sum(y)]).reshape(2, 2)
confusion_df = pd.DataFrame(table, ['𝑎(𝑥)=+1', '𝑎(𝑥)=−1'], ['𝑦=+1', '𝑦=−1'])
confusion_df

,𝑦=+1,𝑦=−1
𝑎(𝑥)=+1,5,5
𝑎(𝑥)=−1,5,5


In [143]:
Precission = confusion_df.iloc[0, 0]/(confusion_df.iloc[0, 0] + confusion_df.iloc[0, 1])
Recall = confusion_df.iloc[0, 0]/(confusion_df.iloc[0, 0] + confusion_df.iloc[1, 0])
F = (2*Precission*Recall)/(Precission+Recall)
Precission, Recall, F

(0.5, 0.5, 0.5)

**Вот видим ,что веса меньше, а Precission и Recall при относительном равенстве accuracy лучше** 